## :star: GenET predict module - DeepPrime

GenET의 predict module은 CRISPR-Cas system들의 gRNA 효율을 예측하는 deep learning model을 손쉽게 사용할 수 있는 함수들을 제공합니다.
이 중, DeepPrime은 prime editing과 관련된 prime editing gRNA (pegRNA)의 효율을 예측하는 모델입니다. 
DeepPrime은 크게 아래의 class들을 불러와서 사용할 수 있습니다. 

- DeepPrime
- DeepPrimeGuideRNA
- DeepPrimeOff

아래의 예시 코드를 이용해서 원하는 기능들을 사용해보세요!

### 💾 Install GenET
---
우선 아래의 기능들을 사용하기 위해서, 현재 environment에 `genet`을 설치합니다. 터미널에 아래의 명령어로 설치할 수 있습니다.

```bash
$ conda create -n genet python=3.9
$ conda activate genet
$ pip install genet
```

추가로, genet.prediction module을 사용하기 위해서는 PyTorch와 ViennaRNA를 추가로 설치해줘야 합니다. 

```bash
$ pip install torch==1.11.0+cu113 --extra-index-url https://download.pytorch.org/whl/cu113
$ conda install viennarna
```

### 🔥 DeepPrime
---
class `DeepPrime`은 target sequence를 기준으로 만들 수 있는 모든 pegRNA를 자동으로 디자인해주고, 각각의 predicted efficiency를 계산해준다.
우선 만들고자 하는 sequence 정보가 담긴 target (unedited and prime-edited) sequence를 준비하고, 모든 pegRNA를 디자인한다.

DeepPrime은 각 pegRNA들을 설계함과 동시에 DeepPrime model에 필요한 biofeatures (Tm, GC counts, MFE ...)를 함께 계산해준다. 이러한 biofeatures를 포함한 전체 데이터를 확인하고 싶다면 `.features`를 사용해서 pd.DataFrame 형태로 불러올 수 있다. 

In [1]:
from genet.predict import DeepPrime

seq_wt   = 'ATGACAATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGAAACTGAGAAGAACTATAACCTGCAAATGTCAACTGAAACCTTAAAGTGAGTATTTAATTGAGCTGAAGT'
seq_ed   = 'ATGACAATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGAAACTGAGACGAACTATAACCTGCAAATGTCAACTGAAACCTTAAAGTGAGTATTTAATTGAGCTGAAGT'

pegrna = DeepPrime('SampleName', seq_wt, seq_ed, edit_type='sub', edit_len=1)

# check designed pegRNAs and biofeatures
pegrna.features.head()

,ID,Spacer,RT-PBS,PBS_len,RTT_len,RT-PBS_len,Edit_pos,Edit_len,RHA_len,Target,...,deltaTm_Tm4-Tm2,GC_count_PBS,GC_count_RTT,GC_count_RT-PBS,GC_contents_PBS,GC_contents_RTT,GC_contents_RT-PBS,MFE_RT-PBS-polyT,MFE_Spacer,DeepSpCas9_score
0,SampleName,AAGACAACACCCTTGCCTTG,CGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGG,7,35,42,34,1,1,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...,...,-340.104846,5,16,21,71.428571,45.714286,50.000000,-10.4,-0.6,45.967537
1,SampleName,AAGACAACACCCTTGCCTTG,CGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGGG,8,35,43,34,1,1,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...,...,-340.104846,6,16,22,75.000000,45.714286,51.162791,-10.4,-0.6,45.967537
2,SampleName,AAGACAACACCCTTGCCTTG,CGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGGGT,9,35,44,34,1,1,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...,...,-340.104846,6,16,22,66.666667,45.714286,50.000000,-10.4,-0.6,45.967537
3,SampleName,AAGACAACACCCTTGCCTTG,CGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGGGTG,10,35,45,34,1,1,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...,...,-340.104846,7,16,23,70.000000,45.714286,51.111111,-10.4,-0.6,45.967537
4,SampleName,AAGACAACACCCTTGCCTTG,CGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGGGTGT,11,35,46,34,1,1,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...,...,-340.104846,7,16,23,63.636364,45.714286,50.000000,-10.4,-0.6,45.967537


이제 각 pegRNA들의 editing efficiency를 예측하고 싶다면, `.predict`를 사용한다. 
각 deep learning model은 training된 모델의 최종 파일 (.pt)이 필요하며, 이는 처음 genet을 설치할 때에는 패키지의 경량화를 위해 설치되지 않는다. 
하지만 만약 genet을 이용해서 모델을 불러오거나 해당 모델을 사용하는 함수를 불러오면, 자동으로 genet이 설치된 경로에 model 파일을 다운로드 받는다. 

우선 아래의 예시는 PE2max를 HEK293T cell line에서 사용했을 때의 pegRNA 효율을 예측한 것이다.
모델 파일이 설치되는 메세지가 나타나지 않았는데, 이는 tutorial 작성 전에 이미 DeepPrime-FT: HEK293T-PE2max 모델을 설치해두었기 때문이다. 

In [2]:
pe2max_output = pegrna.predict(pe_system='PE2max', cell_type='HEK293T')

pe2max_output.head()

,ID,PE2max_score,Spacer,RT-PBS,PBS_len,RTT_len,RT-PBS_len,Edit_pos,Edit_len,RHA_len,Target
0,SampleName,0.904387,AAGACAACACCCTTGCCTTG,CGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGG,7,35,42,34,1,1,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...
1,SampleName,2.375938,AAGACAACACCCTTGCCTTG,CGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGGG,8,35,43,34,1,1,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...
2,SampleName,2.612380,AAGACAACACCCTTGCCTTG,CGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGGGT,9,35,44,34,1,1,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...
3,SampleName,3.641537,AAGACAACACCCTTGCCTTG,CGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGGGTG,10,35,45,34,1,1,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...
4,SampleName,3.768321,AAGACAACACCCTTGCCTTG,CGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGGGTGT,11,35,46,34,1,1,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...


만약, PE2max가 아니라 PE4max-e (PE2max + MMR knockdown + epegRNA system)을 사용한 경우를 예측하고 싶다면, 아래와 같이 옵션을 조정한다.

추가로 더 사용할 수 있는 옵션들에 대해서 확인하고 싶다면, [documentation](https://goosang-yu.github.io/genet/1_Predict/4_predict_pe/#predicting-efficiencies-of-existing-pegrnas)을 확인하라.

DeepPrime-FT: HEK293T-PE4max-e를 처음 불러오는 상황이기 떄문에, 먼저 모델 파일을 다운로드한 후 함수가 실행된다. 한번 다운로드 된 모델 파일은 같은 (가상) 환경에서는 추가로 다운로드 할 필요가 없다. 파일을 다운로드 하는 속도는 사용자의 컴퓨터와 인터넷 상태에 따라 다르다.

In [3]:
pe4max_e_output = pegrna.predict(pe_system='PE4max-e', cell_type='HEK293T')

pe4max_e_output.head()

The model DeepPrime/DP_variant_293T_PE4max_epegRNA_Opti_220428 is not installed. Download checkpoint files.



Downloading: 0KB [00:00, ?KB/s]


File downloaded successfully: /home/gsyu/projects/genet/genet/models/DeepPrime/DP_variant_293T_PE4max_epegRNA_Opti_220428/__init__.py


Downloading: 1KB [00:00, 1319.79KB/s]


File downloaded successfully: /home/gsyu/projects/genet/genet/models/DeepPrime/DP_variant_293T_PE4max_epegRNA_Opti_220428/dp_mean.csv


...

Downloading: 1KB [00:00, 1161.21KB/s]


File downloaded successfully: /home/gsyu/projects/genet/genet/models/DeepPrime/DP_variant_293T_PE4max_epegRNA_Opti_220428/mean_231124.csv


Downloading: 1KB [00:00, 3429.52KB/s]


File downloaded successfully: /home/gsyu/projects/genet/genet/models/DeepPrime/DP_variant_293T_PE4max_epegRNA_Opti_220428/std_231124.csv


,ID,PE4max-e_score,Spacer,RT-PBS,PBS_len,RTT_len,RT-PBS_len,Edit_pos,Edit_len,RHA_len,Target
0,SampleName,2.095974,AAGACAACACCCTTGCCTTG,CGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGG,7,35,42,34,1,1,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...
1,SampleName,4.202025,AAGACAACACCCTTGCCTTG,CGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGGG,8,35,43,34,1,1,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...
2,SampleName,5.702226,AAGACAACACCCTTGCCTTG,CGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGGGT,9,35,44,34,1,1,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...
3,SampleName,5.970952,AAGACAACACCCTTGCCTTG,CGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGGGTG,10,35,45,34,1,1,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...
4,SampleName,6.580553,AAGACAACACCCTTGCCTTG,CGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGGGTGT,11,35,46,34,1,1,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...


### 🔥 DeepPrimeGuideRNA
---
class `DeepPrimeGuideRNA`는 이미 정해진 pegRNA와 target에 대해서 prime editing efficiency를 예측하기 위한 기능을 제공한다. 우선 아래와 같이 불러온다.

In [4]:
from genet.predict import DeepPrimeGuideRNA

Target sequence는 정확히 74nt DNA sequence를 입력해야 하며, 아래와 같이 특정한 위치가 고정되어 있다. 
- Position 5-24: Protospacer
- Position 25-27: PAM

그 외에 edit length, position, and type에 대한 정확한 정보가 함께 입력되어야 한다. 아래는 이를 사용하는 예시이다. 

`DeepPrime`과 마찬가지로, `.features`을 사용해서 입력된 pegRNA에 대한 feature 정보가 담긴 DataFrame을 볼 수 있다. 

In [6]:
target    = 'ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGAAACTGAGAAGAACTATAACCTGCAAATG'
pbs       = 'GGCAAGGGTGT'
rtt       = 'CGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAA'
edit_len  = 1
edit_pos  = 34
edit_type = 'sub'

pegrna = DeepPrimeGuideRNA('pegRNA_test', target=target, pbs=pbs, rtt=rtt, 
                           edit_len=edit_len, edit_pos=edit_pos, edit_type=edit_type)

pegrna.features

,ID,Spacer,RT-PBS,PBS_len,RTT_len,RT-PBS_len,Edit_pos,Edit_len,RHA_len,Target,...,deltaTm_Tm4-Tm2,GC_count_PBS,GC_count_RTT,GC_count_RT-PBS,GC_contents_PBS,GC_contents_RTT,GC_contents_RT-PBS,MFE_RT-PBS-polyT,MFE_Spacer,DeepSpCas9_score
0,pegRNA_test,AAGACAACACCCTTGCCTTG,CGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGGGTGT,11,35,46,34,1,1,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...,...,-340.104846,7,16,23,63.636364,45.714286,50.0,-10.4,-0.6,45.967541


`DeepPrimeGuideRNA`에서 예측 값은 `.predict`를 사용해서 얻을 수 있다. `DeepPrime.predict`와 다르게, `DeepPrimeGuideRNA.predict`는 예측 값이 단일 값으로 반환된다는 것을 주의하라. 

In [7]:
pe2max_score = pegrna.predict('PE2max')

pe2max_score

3.768320083618164